In [21]:
import uuid
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import my_connector
from datetime import datetime
from my_connector import MySQLConnector

In [22]:
df = pd.read_csv("Datasets/Data_Combinada.csv", low_memory=False)

In [23]:
df.head(10)

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,numero_pasajeros,fugado
0,2022S000001,01/01/2022,1:30:00,"AVDA. ALBUFERA, 19",19,13.0,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,...,De 30 a 34 años,M,NaN,NaN,"443359,226","4472082,272",0.0,0.0,2,No
1,2022S000001,01/01/2022,1:30:00,"AVDA. ALBUFERA, 19",19,13.0,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,...,De 45 a 49 años,H,NaN,NaN,"443359,226","4472082,272",0.0,0.0,2,No
2,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,NaN,Motocicleta hasta 125cc,...,De 30 a 34 años,H,NaN,NaN,"441155,351","4474129,588",1.0,0.0,3,No
3,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,NaN,Motocicleta hasta 125cc,...,De 35 a 39 años,M,NaN,NaN,"441155,351","4474129,588",0.0,0.0,3,No
4,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,NaN,Turismo,...,De 40 a 44 años,H,NaN,NaN,"441155,351","4474129,588",0.0,0.0,3,No
5,2022S000003,01/01/2022,1:50:00,"CALL. SAN BERNARDO, 53",53,1.0,CENTRO,Atropello a persona,Despejado,Motocicleta > 125cc,...,Desconocido,H,NaN,NaN,"439995,351","4475212,523",0.0,0.0,2,No
6,2022S000003,01/01/2022,1:50:00,"CALL. SAN BERNARDO, 53",53,1.0,CENTRO,Atropello a persona,Despejado,Motocicleta > 125cc,...,De 18 a 20 años,M,7.0,Asistencia sanitaria sólo en el lugar del acci...,"439995,351","4475212,523",0.0,0.0,2,No
7,2022S000004,01/01/2022,3:40:00,CALL. ALCALA / PLAZA. EISENHOWER,728,20.0,SAN BLAS-CANILLEJAS,Choque contra obstáculo fijo,Despejado,Turismo,...,De 50 a 54 años,H,2.0,Ingreso inferior o igual a 24 horas,"449693,925","4477837,552",0.0,0.0,2,No
8,2022S000004,01/01/2022,3:40:00,CALL. ALCALA / PLAZA. EISENHOWER,728,20.0,SAN BLAS-CANILLEJAS,Choque contra obstáculo fijo,Despejado,Turismo,...,De 55 a 59 años,M,3.0,Ingreso superior a 24 horas,"449693,925","4477837,552",0.0,0.0,2,No
9,2022S000005,01/01/2022,4:30:00,"AUTOV. A-42, +00500E",+00500E,12.0,USERA,Choque contra obstáculo fijo,Despejado,Turismo,...,De 45 a 49 años,H,NaN,NaN,"438952,303","4469985,021",1.0,0.0,2,No


-- Creación de la tabla de Distritos
CREATE TABLE Distritos (
    distrito_id INT PRIMARY KEY,
    codigo_distrito VARCHAR(255),
    nombre_distrito VARCHAR(255)
);

-- Creación de la tabla de Direcciones
CREATE TABLE Direcciones (
    direccion_id INT PRIMARY KEY,
    calle VARCHAR(255),
    numero INT
);

-- Creación de la tabla de Accidentes
CREATE TABLE Accidentes (
    num_expediente VARCHAR(255) PRIMARY KEY,
    fecha DATE,
    hora TIME,
    direccion_id INT,
    distrito_id INT,
    tipo_accidente VARCHAR(255),
    estado_meteorologico VARCHAR(255),
    coordenada_x_utm VARCHAR(255),
    coordenada_y_utm VARCHAR(255),
    positivo_alcohol BOOLEAN,
    positivo_droga BOOLEAN,
    fugado BOOLEAN,
    FOREIGN KEY (direccion_id) REFERENCES Direcciones(direccion_id),
    FOREIGN KEY (distrito_id) REFERENCES Distritos(distrito_id)
);

-- Creación de la tabla de Personas Involucradas en Accidentes
CREATE TABLE PersonasInvolucradas (
    id INT PRIMARY KEY,
    num_expediente VARCHAR(255),
    tipo_vehiculo VARCHAR(255),
    tipo_persona VARCHAR(255),
    rango_edad VARCHAR(255),
    sexo CHAR(1),
    codigo_lesividad INT,
    lesividad VARCHAR(255),
    numero_pasajeros INT,
    FOREIGN KEY (num_expediente) REFERENCES Accidentes(num_expediente)
);


Calculamos el máximo para dimensionar correctamente la base de datos

In [24]:
df['num_expediente'].max()

'2023S029136'

In [25]:
connector = MySQLConnector('localhost', 'root', 'root', 'TNT')
connector.connect()

Connected to MySQL database.


In [26]:
cod_distrito_unique = df['cod_distrito'].unique()
distrito_unique = df['distrito'].unique()
emparejados = {cod: dist for cod, dist in zip(cod_distrito_unique, distrito_unique)}

In [41]:
for cod, dist in emparejados.items():
    distrito_query = f"""
        INSERT INTO Distritos (codigo_distrito, nombre_distrito) VALUES ('{cod}', '{dist}')
        """

    connector.execute_query(distrito_query)

distrito_query = f"""
        INSERT INTO Distritos (codigo_distrito, nombre_distrito) VALUES ('0', 'DESCONOCIDO')
        """
connector.execute_query(distrito_query)

Error:  1062 (23000): Duplicate entry '13' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '3' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '1' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '20' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '12' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '7' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '14' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '10' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '16' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '2' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '11' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '6' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '15' for key 'Distritos.PRIMARY'
Error:  1062 (23000): Duplicate entry '17' for key 'Distritos.PRIMARY'
Error:  106

In [28]:
df

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,numero_pasajeros,fugado
0,2022S000001,01/01/2022,1:30:00,"AVDA. ALBUFERA, 19",19,13.0,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,...,De 30 a 34 años,M,NaN,NaN,"443359,226","4472082,272",0.0,0.0,2,No
1,2022S000001,01/01/2022,1:30:00,"AVDA. ALBUFERA, 19",19,13.0,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,...,De 45 a 49 años,H,NaN,NaN,"443359,226","4472082,272",0.0,0.0,2,No
2,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,NaN,Motocicleta hasta 125cc,...,De 30 a 34 años,H,NaN,NaN,"441155,351","4474129,588",1.0,0.0,3,No
3,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,NaN,Motocicleta hasta 125cc,...,De 35 a 39 años,M,NaN,NaN,"441155,351","4474129,588",0.0,0.0,3,No
4,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,NaN,Turismo,...,De 40 a 44 años,H,NaN,NaN,"441155,351","4474129,588",0.0,0.0,3,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74956,2023S028337,31/08/2023,23:20:00,"CALL. GOLFO DE SALONICA, 12",12,16.0,HORTALEZA,Otro,Despejado,Turismo,...,De 55 a 59 años,M,14.0,Sin asistencia sanitaria,443580.365,4480912.376,0.0,0.0,3,No
74957,2023S028341,31/08/2023,22:30:00,CALLE SAN SERAPIO,1,12.0,USERA,Alcance,Despejado,Motocicleta hasta 125cc,...,De 30 a 34 años,H,7.0,Asistencia sanitaria sólo en el lugar del acci...,440795.55,4471044.789,0.0,0.0,2,No
74958,2023S028341,31/08/2023,22:30:00,CALLE SAN SERAPIO,1,12.0,USERA,Alcance,Despejado,Turismo,...,De 35 a 39 años,H,14.0,Sin asistencia sanitaria,440795.55,4471044.789,0.0,0.0,2,No
74959,2023S028352,31/08/2023,14:40:00,"AUTOV. M-30, 00NC06",00NC06,5.0,CHAMARTÍN,Caída,Despejado,Motocicleta hasta 125cc,...,De 40 a 44 años,H,1.0,Atención en urgencias sin posterior ingreso,442825.5,4481003.193,0.0,0.0,1,No


In [29]:
df['numero'].fillna('S/N', inplace=True)
df['tipo_vehiculo'].fillna('Desconocido', inplace=True)
df['lesividad'].fillna('Sin asistencia - Desconocido', inplace=True)
df['cod_lesividad'].fillna(0, inplace=True)

In [30]:
df

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,numero_pasajeros,fugado
0,2022S000001,01/01/2022,1:30:00,"AVDA. ALBUFERA, 19",19,13.0,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,...,De 30 a 34 años,M,0.0,Sin asistencia - Desconocido,"443359,226","4472082,272",0.0,0.0,2,No
1,2022S000001,01/01/2022,1:30:00,"AVDA. ALBUFERA, 19",19,13.0,PUENTE DE VALLECAS,Alcance,Despejado,Turismo,...,De 45 a 49 años,H,0.0,Sin asistencia - Desconocido,"443359,226","4472082,272",0.0,0.0,2,No
2,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,NaN,Motocicleta hasta 125cc,...,De 30 a 34 años,H,0.0,Sin asistencia - Desconocido,"441155,351","4474129,588",1.0,0.0,3,No
3,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,NaN,Motocicleta hasta 125cc,...,De 35 a 39 años,M,0.0,Sin asistencia - Desconocido,"441155,351","4474129,588",0.0,0.0,3,No
4,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,NaN,Turismo,...,De 40 a 44 años,H,0.0,Sin asistencia - Desconocido,"441155,351","4474129,588",0.0,0.0,3,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74956,2023S028337,31/08/2023,23:20:00,"CALL. GOLFO DE SALONICA, 12",12,16.0,HORTALEZA,Otro,Despejado,Turismo,...,De 55 a 59 años,M,14.0,Sin asistencia sanitaria,443580.365,4480912.376,0.0,0.0,3,No
74957,2023S028341,31/08/2023,22:30:00,CALLE SAN SERAPIO,1,12.0,USERA,Alcance,Despejado,Motocicleta hasta 125cc,...,De 30 a 34 años,H,7.0,Asistencia sanitaria sólo en el lugar del acci...,440795.55,4471044.789,0.0,0.0,2,No
74958,2023S028341,31/08/2023,22:30:00,CALLE SAN SERAPIO,1,12.0,USERA,Alcance,Despejado,Turismo,...,De 35 a 39 años,H,14.0,Sin asistencia sanitaria,440795.55,4471044.789,0.0,0.0,2,No
74959,2023S028352,31/08/2023,14:40:00,"AUTOV. M-30, 00NC06",00NC06,5.0,CHAMARTÍN,Caída,Despejado,Motocicleta hasta 125cc,...,De 40 a 44 años,H,1.0,Atención en urgencias sin posterior ingreso,442825.5,4481003.193,0.0,0.0,1,No


In [31]:
df = df.sort_values(by=['fecha', 'hora']).ffill()

In [32]:
df = df.sort_values(by=['fecha', 'hora']).bfill()

In [33]:
df['fugado'].replace({'No': 0, 'Si': 1}, inplace=True)

In [34]:
df

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,numero_pasajeros,fugado
13,2022S000008,01/01/2022,0:25:00,"PLAZA. ENCUENTRO, 2",2,14.0,MORATALAZ,Choque contra obstáculo fijo,Despejado,Turismo,...,De 21 a 24 años,M,0.0,Sin asistencia - Desconocido,"444759,786","4472970,631",0.0,0.0,1,0
2,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,Despejado,Motocicleta hasta 125cc,...,De 30 a 34 años,H,0.0,Sin asistencia - Desconocido,"441155,351","4474129,588",1.0,0.0,3,0
3,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,Despejado,Motocicleta hasta 125cc,...,De 35 a 39 años,M,0.0,Sin asistencia - Desconocido,"441155,351","4474129,588",0.0,0.0,3,0
4,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO / PASEO. PRADO,2,3.0,RETIRO,Colisión fronto-lateral,Despejado,Turismo,...,De 40 a 44 años,H,0.0,Sin asistencia - Desconocido,"441155,351","4474129,588",0.0,0.0,3,0
165,2022S000088,01/01/2022,11:45:00,AVDA. VALLADOLID / PLAZA. REPUBLICA DE CHILE,10,9.0,MONCLOA-ARAVACA,Colisión lateral,Despejado,Turismo,...,De 30 a 34 años,M,0.0,Sin asistencia - Desconocido,"437731,276","4476176,535",0.0,0.0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44973,2022S041231,31/12/2022,6:25:00,"C/ JOSE ORTEGA Y GASSET, 40-42",40,4.0,SALAMANCA,Choque contra obstáculo fijo,Despejado,Turismo,...,De 30 a 34 años,H,0.0,Sin asistencia - Desconocido,"442441,878","4475706,049",0.0,0.0,3,0
44974,2022S041231,31/12/2022,6:25:00,"C/ JOSE ORTEGA Y GASSET, 40-42",40,4.0,SALAMANCA,Choque contra obstáculo fijo,Despejado,Turismo,...,Desconocido,H,0.0,Sin asistencia - Desconocido,"442441,878","4475706,049",0.0,0.0,3,0
44975,2022S041231,31/12/2022,6:25:00,"C/ JOSE ORTEGA Y GASSET, 40-42",40,4.0,SALAMANCA,Choque contra obstáculo fijo,Despejado,Turismo,...,De 45 a 49 años,H,0.0,Sin asistencia - Desconocido,"442441,878","4475706,049",0.0,0.0,3,0
45018,2022S041262,31/12/2022,6:50:00,"CALL. ENRIQUE GARCIA ALVAREZ, 11",11,18.0,VILLA DE VALLECAS,Choque contra obstáculo fijo,Despejado,Turismo,...,De 35 a 39 años,H,0.0,Sin asistencia - Desconocido,"447813,397","4470373,787",0.0,0.0,1,0


In [35]:
df['localizacion'] = df['localizacion'].str.replace("/", '-').str.replace("'", "")
df['coordenada_y_utm'] = df['coordenada_y_utm'].str.replace(",", '.')
df['coordenada_x_utm'] = df['coordenada_x_utm'].str.replace(",", '.')

In [36]:
accidentes = df.filter(
    items=['num_expediente', 'fecha', 'hora', 'localizacion', 'numero', 'cod_distrito', 'tipo_accidente',
           'estado_meteorológico', 'coordenada_x_utm', 'coordenada_y_utm', 'positiva_alcohol', 'positiva_droga',
           'fugado'])
accidentes.head()

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,tipo_accidente,estado_meteorológico,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,fugado
13,2022S000008,01/01/2022,0:25:00,"PLAZA. ENCUENTRO, 2",2,14.0,Choque contra obstáculo fijo,Despejado,444759.786,4472970.631,0.0,0.0,0
2,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO - PASEO. PRADO,2,3.0,Colisión fronto-lateral,Despejado,441155.351,4474129.588,1.0,0.0,0
3,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO - PASEO. PRADO,2,3.0,Colisión fronto-lateral,Despejado,441155.351,4474129.588,0.0,0.0,0
4,2022S000002,01/01/2022,0:30:00,PLAZA. CANOVAS DEL CASTILLO - PASEO. PRADO,2,3.0,Colisión fronto-lateral,Despejado,441155.351,4474129.588,0.0,0.0,0
165,2022S000088,01/01/2022,11:45:00,AVDA. VALLADOLID - PLAZA. REPUBLICA DE CHILE,10,9.0,Colisión lateral,Despejado,437731.276,4476176.535,0.0,0.0,0


In [37]:
patron = r'\d{2}/\d{2}/\d{4}'

filas_con_formato_incorrecto = df[~df['fecha'].str.match(patron)]

In [38]:
filas_con_formato_incorrecto

,num_expediente,fecha,hora,localizacion,numero,cod_distrito,distrito,tipo_accidente,estado_meteorológico,tipo_vehiculo,...,rango_edad,sexo,cod_lesividad,lesividad,coordenada_x_utm,coordenada_y_utm,positiva_alcohol,positiva_droga,numero_pasajeros,fugado


In [39]:
accidentes = accidentes.drop_duplicates(subset=['num_expediente'])

In [40]:
i = 0
for index, row in accidentes.iterrows():
    i = i + 1
    try:
        sql = f"""
        INSERT INTO Accidentes (num_expediente, fecha, hora, calle, numero, codigo_distrito, tipo_accidente, estado_meteorologico, coordenada_x_utm, coordenada_y_utm, positivo_alcohol, positivo_droga, fugado)
        VALUES ('{row['num_expediente']}', '{datetime.strptime(row['fecha'], "%d/%m/%Y")}', '{datetime.strptime(row['hora'], "%H:%M:%S")}', '{row['localizacion']}', '{row['numero']}', {row['cod_distrito']}, '{row['tipo_accidente']}', '{row['estado_meteorológico']}', '{float(row['coordenada_x_utm'])}', '{float(row['coordenada_y_utm'])}', {row['positiva_alcohol']}, {row['positiva_droga']}, {row['fugado']});
    """
        print(f"Current row index: {index}, count: {i}")
        connector.execute_query(sql)
    except Exception as e:
        print(e)

Current row index: 13, count: 1
Query executed successfully.
Current row index: 2, count: 2
Query executed successfully.
Current row index: 165, count: 3
Query executed successfully.
Current row index: 74, count: 4
Query executed successfully.
Current row index: 70, count: 5
Query executed successfully.
Current row index: 72, count: 6
Query executed successfully.
Current row index: 33, count: 7
Query executed successfully.
Current row index: 37, count: 8
Query executed successfully.
Current row index: 45, count: 9
Query executed successfully.
Current row index: 43, count: 10
Query executed successfully.
Current row index: 41, count: 11
Query executed successfully.
Current row index: 48, count: 12
Query executed successfully.
Current row index: 16, count: 13
Query executed successfully.
Current row index: 51, count: 14
Query executed successfully.
Current row index: 49, count: 15
Query executed successfully.
Current row index: 0, count: 16
Query executed successfully.
Current row index: